<a href="https://colab.research.google.com/github/Namgyoung/Machine_Learning/blob/main/AI_Finance_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#stock return
import numpy as np
import pandas as pd

data = np.random.rand(1,100)    #.rand는 0부터 1사이의 균일 분포에서 난수 matrix array 생성, (1,100)은 (1행,100열)matrix 의미
data

In [ ]:
df_data = pd.DataFrame(data.T)
df_data

In [ ]:
stock_return = (df_data - df_data.shift(1))/df_data.shift(1)      #p(t)-p(t-1)/p(t-1). return 구하는 공식 적용
stock_return

In [ ]:
#log return
stock_return = np.log(df_data) - np.log(df_data.shift(1))
stock_return

In [ ]:
stock_return = df_data.pct_change()   #percentage change
stock_return

In [ ]:
import matplotlib.pyplot as plt
plt.hist(stock_return.dropna());

In [ ]:
import seaborn as sns
tips = sns.load_dataset('tips')
tips

In [ ]:
np.cov(tips.tip, tips.total_bill)   #(2,2)matrix에서 (1,1), (2,2): Varaince by itself / (1,2),(2,1): Covariance

In [ ]:
print('Correlation coefficient is {:.4f}'.format(tips.tip.corr(tips.total_bill)))   #tip이랑 total_bill 사이의 correlation

In [ ]:
tips.corr()

In [ ]:
sns.heatmap(tips.corr(), annot=True)
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(tips.corr(), annot=True, vmin=-1, vmax=1, center=0)
plt.show()

In [83]:
import statsmodels.api as sm
import plotly.graph_objs as go
import plotly

In [84]:
n_assets = 5
n_simulation = 500

In [ ]:
returns = np.random.randn(n_assets,n_simulation)
returns

In [86]:
random = np.random.rand(n_assets)
weights = random / len(random)
def exp_port_return(returns):
  rets = np.mean(returns, axis=1)
  cov = np.cov(rets.T, aweights=weights, ddof=0)
  portfolio_return = np.dot(weights, rets.T)
  portfolio_std_dev = np.sqrt(np.dot(weights, np.dot(cov, weights)))
  return portfolio_return, portfolio_std_dev

In [87]:
portfolio_return, portfolio_std_dev = exp_port_return(returns)

In [ ]:
print(portfolio_return)
print(portfolio_std_dev)

In [ ]:
portfolio = np.array([exp_port_return(np.random.randn(5, i)) for i in range(1,101)])
portfolio

In [ ]:
best_fit = sm.OLS(portfolio[:,1], sm.add_constant(portfolio[:,0])).fit().fittedvalues
best_fit

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(name='Risk-Return Relationship', 
                         x=portfolio[:,0], y=portfolio[:,1], 
                         mode='markers'))
fig.add_trace(go.Scatter(name='best fit line', x=np.array(portfolio[:,0]), 
                                               y=best_fit, mode='lines'))
fig.update_layout(xaxis_title='Mean',
                  yaxis_title='Standard Deviation')
fig.show()

In [ ]:
#Yahoo Finance
!pip install yfinance
import yfinance as yf

In [93]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [ ]:
tickers = ['^GSPC', 'AAPL', 'MSFT', 'INTC']   #^GSPC는 인덱스임
start = '2010-01-01'
end = '2021-01-01'
stock_prices = yf.download(tickers, start=start, end=end, interval='1d')    #1d는 1day
stock_prices.head()

In [ ]:
stock_prices = stock_prices['Close']
stock_prices.head()

In [ ]:
plt.plot(stock_prices.iloc[:, :-1])
plt.title('S&P 500 listed stock prices')
plt.xlabel('Date')
plt.ylabel('Prices')
plt.show()

In [ ]:
#Fred API
!pip3 install fredapi

In [99]:
from fredapi import Fred
fred = Fred(api_key='78b14ec6ba46f484b94db43694468bb1')

In [ ]:
fred.search('interest')

In [ ]:
interest_rate = fred.get_series('FEDFUNDS')
interest_rate.head()

In [ ]:
plt.plot(interest_rate)
plt.title('Federal Reserve Interest Rate')
plt.xlabel('Date')
plt.ylabel('%')
plt.show()

In [ ]:
#Quandl
!pip3 install quandl

In [113]:
import quandl
crude_oil = quandl.get('FRED/DCOILBRENTEU', authtoken='vEjGTysiCFBuN-z5bjGP')

In [ ]:
crude_oil.head()

In [ ]:
plt.plot(crude_oil)
plt.title('Crude Oil Prices')
plt.xlabel('Date')
plt.ylabel('$')
plt.show()

In [ ]:
#World Bank Data
!pip3 install world_bank_data

In [117]:
import world_bank_data as wb

In [ ]:
wb.get_sources()

In [ ]:
wb.search_indicators('', source=2)    #source2는 id를 의미 (여기서는 WDI)

In [ ]:
wb.search_indicators('inflation', source=2)

In [ ]:
wb.search_countries('Korea')


In [ ]:
inflation = wb.get_series('FP.CPI.TOTL.ZG', country='KOR', date='2010:2020', id_or_value='id', simplify_index=True)
inflation

In [ ]:
unemployment = wb.get_series('SL.UEM.TOTL.NE.ZS', country=['KOR', 'USA'], date='2010:2020', id_or_value='id', simplify_index=True)
unemployment

In [130]:
#Capital Asset Pricing Model (CAPM)
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime

In [132]:
stocks = ['AAPL', 'IBM', 'MSFT', 'INTC', '^GSPC']
start = '2021-1-1'
end = '2022-1-1'
stock_prices = yf.download(stocks, start=start, end=end, interval='1d')
stock_prices.head()

[*********************100%***********************]  5 of 5 completed


Adj Close                         ...    Volume                      
                  AAPL         IBM       INTC  ...      INTC      MSFT       ^GSPC
Date                                           ...                                
2021-01-04  128.617111  112.813461  48.414623  ...  46102500  37130100  5006680000
2021-01-05  130.207291  114.815948  49.330872  ...  24866600  23823000  4582620000
2021-01-06  125.824326  117.683174  49.808479  ...  36809600  35930700  6049970000
2021-01-07  130.117844  117.410103  50.870934  ...  32092300  27694500  5080870000
2021-01-08  131.240921  116.991394  50.344585  ...  34625400  22956200  4764180000

[5 rows x 30 columns]

In [133]:
stock_prices = stock_prices['Close']

In [134]:
stock_prices.isnull().sum()     #빈 셀이 있나 확인차원에서 진행

AAPL     0
IBM      0
INTC     0
MSFT     0
^GSPC    0
dtype: int64

In [135]:
stock_prices.describe()

,AAPL,IBM,INTC,MSFT,^GSPC
count,252.000000,252.000000,252.000000,252.000000,252.000000
mean,140.989365,128.959140,55.851786,275.940754,4273.385635
std,14.659580,8.428465,4.543209,36.940247,287.443815
min,116.360001,113.365204,47.889999,212.250000,3700.649902
25%,128.962498,121.572191,52.995000,244.097496,4071.214966
50%,141.305000,131.080307,55.225000,277.165009,4304.000000
75%,148.962505,136.113766,57.990002,301.842491,4486.287476
max,180.330002,144.627151,68.260002,343.109985,4793.060059


In [137]:
stock_prices = stock_prices.rename({'^GSPC':'GSPC'}, axis='columns')

In [138]:
from fredapi import Fred
fred = Fred(api_key='78b14ec6ba46f484b94db43694468bb1')

In [ ]:
fred.search('risk free')

In [142]:
risk_free = fred.get_series('DGS1MO')
risk_free['2021-01-01':'2022-01-01']

2021-01-01     NaN
2021-01-04    0.09
2021-01-05    0.08
2021-01-06    0.09
2021-01-07    0.09
              ... 
2021-12-27    0.04
2021-12-28    0.03
2021-12-29    0.01
2021-12-30    0.06
2021-12-31    0.06
Length: 261, dtype: float64